In [17]:
import fake_useragent
import requests
import json
from  bs4 import BeautifulSoup
import pprint
import pandas as pd
import numpy as np
from datetime import datetime
import re

In [18]:
def get_useragent():
    ua = fake_useragent.UserAgent()
    return ua.random


In [19]:

def get_onepage(url, params, headers):
    r = requests.get(url, params=params, headers=headers)
    r.encoding = 'utf-8'
    json_data = json.loads(r.text)
    return json_data


In [20]:
# (1) 通过正则表达式获取发布时间、标题、标签、链接、转发数、评论数、点赞数、视频播放量
def process_one_page(mydata):
    datas = []
    dist = {}
    for i in range(len(mydata["data"]["list"])):
        if mydata["data"]["list"][i]["text"].find("的微博视频") == -1:
            continue
        text_raw = mydata["data"]["list"][i]["text_raw"]
        title = re.sub(r'(#\S+#)|(\bhttps?://\S+\b)', '',text_raw)
        lable = ''.join(re.findall(r"#.*?#", text_raw))
        url = ''.join(re.findall(r'http://\S+', text_raw))
        created_at = mydata["data"]["list"][i]["created_at"]
        created_at = datetime.strptime(
            created_at, '%a %b %d %H:%M:%S %z %Y').strftime('%Y-%m-%d')
        reposts_count = mydata["data"]["list"][i]["reposts_count"]
        comments_count = mydata["data"]["list"][i]["comments_count"]
        attitudes_count = mydata["data"]["list"][i]["attitudes_count"]
        online_users_number = mydata["data"]["list"][i]["page_info"]["media_info"]["online_users_number"]
        datas.append({
            "created_at": created_at,
            "title": title,
            "lable": lable,
            "url": url,
            "reposts_count": reposts_count,
            "comments_count": comments_count,
            "attitudes_count": attitudes_count,
            "online_users_number": online_users_number
        })
    return datas



In [21]:
def craw_page(number):
    url = "https://weibo.com/ajax/statuses/searchProfile"
    params = {
        #  德国人Leo乐柏说 1104911015
        # 司徒建国Stu 2006222803
        'uid': 2006222803,
        'page': number,
        'feature': 4,
        "starttime": 1609430400,
        "endtime": 1664553600
    }
    headers = {
        'User-Agent': get_useragent(),
        'cookie': "SINAGLOBAL=6008923544490.623.1669600730018; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WWh0Xsq1MAgxu2XAbnI-Sb55JpX5KMhUgL.Foqce0.4eK2X1K-2dJLoIEBLxK-L1h5L1-qLxKML1-BL1h5LxK-L1K2LB.BLxKnLB.qL1K5t; UOR=,,login.sina.com.cn; SCF=Ap2YMJwYeBXVWkX3383x4VI14durTT4lsVde7VL_sz4ywsxV7Jd7tJ4yjA4kDO5HEKhki79hdsJjELj9F_7Gozg.; SUB=_2A25JHBFUDeRhGeBI6FsY8S_IwjmIHXVqaAWcrDV8PUNbmtANLRjwkW9NRp7kGDhEQQd_w24UE_VdlRFzSuPNjSTi; ALF=1681911299; SSOLoginState=1679319300; PC_TOKEN=3bd4e2c9b2; XSRF-TOKEN=P7qjIF6DkD1zAyhSCJXxJmpz; _s_tentry=weibo.com; Apache=3274496889012.7437.1679320773863; ULV=1679320774007:9:6:1:3274496889012.7437.1679320773863:1678619453169; WBPSESS=USfWtr-hzUv9dxm42OaQDLa7_1kqMNlxcjy8i3kHIdISf0sLPydmsc1APmCBGPvwjUOvjmN5XVfv_4-RnAy6LeLd0radFCjspxnG-hFawwhj-VU2vzXpz0P3SgvRrXJ92r6c5f6BVSO1rOA4zna0Lg=="
    }
    data_json = get_onepage(url, params, headers)
    page_datas = process_one_page(data_json)
    return page_datas


In [22]:
all_pages_datas = []
for number in range(1,15):
    all_pages_datas.extend(craw_page(number))

    

In [23]:
df = pd.DataFrame(all_pages_datas)
#df 按照观看数排序
df = df.sort_values(by="online_users_number",ascending=False)
# df 转换为 excel
df.to_excel("司徒建国Stu.xlsx", index=False)

In [24]:
df

,created_at,title,lable,url,reposts_count,comments_count,attitudes_count,online_users_number
49,2021-07-28,【】我在2019年跟随@人民日报体育 记者见到了@举重侯志慧 ，我跟她说以后她一定会变成奥运...,#英国小哥神预言侯志慧夺冠#,http://t.cn/A6feAKFP,267,156,4043,967067
7,2022-07-08,英国小哥翻唱周杰伦 这个版本好英伦！很喜欢 ，我也翻译了英文歌词，弹了英伦风格，希望你们喜...,#最伟大的作品##周杰伦新歌##新星V计划#,http://t.cn/A6aSoON1,29,84,568,244358
54,2021-06-11,[龙舟]英国爸妈第一次吃粽子过端午，博步想吃粽叶？！[甜甜粽] 肉粽枣粽一起来！和我们一家一...,#端午节要这样过##粽子节快乐##端午节#,http://t.cn/A6V32P0I,52,310,1646,155894
13,2022-06-03,好多人想看我爸妈吃粽子，他们吃过了！还用了刀叉吃！太搞笑了 [咸粽党][甜粽党][我最粽意你...,#美好生活守护家#,http://t.cn/A6V32P0I,2,33,187,155894
21,2022-02-26,英国爸妈试吃中国怀旧零食，干脆面yyds一口上瘾！ ​​​,#年味vlog日记##怀旧零食##小浣熊干脆面#,http://t.cn/A66zybds,19,122,593,144593
30,2021-12-27,从成都到重庆再到北京，英国爸妈博步韩梅梅的中国VLOG共12集整合在这里，方便大家观看，谢谢...,#圣诞心愿#,http://t.cn/Aiu9uzGqhttp://t.cn/Ai17zGdn,39,71,452,131300
22,2022-02-16,我爸妈去年第一次吃汤圆果然他们都爱吃 ​​​,#虎年新春迎虎福#,http://t.cn/A6t6OB5W,3,22,153,122160
11,2022-06-10,英国爸妈参加英国最火的答题闯关节目！数学题中国小学生都能答好吗？[666] ​​​,#知识分享大会#,http://t.cn/A6XdYYUH,14,48,208,99352
45,2021-08-17,给英国爸妈寄了中国最有名的糕点 博步眼睛都直了！一起快乐拆箱吧[太开心] ​​​,#吃货在这里##甜品#,http://t.cn/A6IaN9sK,38,200,1213,90599
3,2022-08-04,[男孩儿][女孩儿]老年CP秀英式土味情话！英国爸妈吃中国零食过七夕[桃花][心] ​​​,#爱笑夏令营##七夕##七夕情侣在⼀起要做什么#,http://t.cn/A6asXUct,9,37,298,88439


In [25]:
all_pages_datas

[{'created_at': '2022-09-30',
  'title': '英国爸妈第一次吃叉烧包 吃货老爸撑到爆炸[小黄人微笑]（扶我起来还能吃！）   \u200b\u200b\u200b',
  'lable': '#早茶##美食打卡#',
  'url': 'http://t.cn/A6oZAOla',
  'reposts_count': 27,
  'comments_count': 88,
  'attitudes_count': 581,
  'online_users_number': 49099},
 {'created_at': '2022-09-09',
  'title': '中国最火月饼在英国卖到天价？必须给爸妈整上！跟我们一起过中秋节吧[团圆月饼][月亮代表我的心]     \u200b\u200b\u200b',
  'lable': '#中秋节快乐##月饼##流心奶黄月饼#',
  'url': 'http://t.cn/A6SHq61H',
  'reposts_count': 32,
  'comments_count': 159,
  'attitudes_count': 751,
  'online_users_number': 54896},
 {'created_at': '2022-08-05',
  'title': '[男孩儿][女孩儿]英国爸妈结婚50年还能这么甜！秘诀竟是...[心]   \u200b\u200b\u200b',
  'lable': '#爱笑夏令营#',
  'url': 'http://t.cn/A6SvV3Db',
  'reposts_count': 17,
  'comments_count': 42,
  'attitudes_count': 368,
  'online_users_number': 34716},
 {'created_at': '2022-08-04',
  'title': '[男孩儿][女孩儿]老年CP秀英式土味情话！英国爸妈吃中国零食过七夕[桃花][心]     \u200b\u200b\u200b',
  'lable': '#爱笑夏令营##七夕##七夕情侣在⼀起要做什么#',
  'url': 'http://t.cn/A6asXUct',
 

In [26]:
excle_name = "司徒建国Stu"
"{}.xlsx".format(excle_name)

'司徒建国Stu.xlsx'